In [ ]:
!pip install pulp

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd
import pulp as p
import matplotlib.pyplot as plt
plt.style.use('seaborn-whitegrid')
import numpy as np
n_c=23 #number of customers
n_f=23 #number of facilities


#Create activation cost f for the facilities

f=[600.00,528.17,730.43,606.06,1200.00,4166.67,833.33,666.67,1200.00,1111.11,625.00,1200.00,833.33,1450.00,568.18,1041.67,1653.85,1285.71,1000.00,750.00,965.52,550.00,923.08]

col_names = ['col'+ str(i) for i in range(0,23)]
df = pd.read_csv('Matrix.csv', names=col_names)
df.tail()
df=df.to_numpy()

c=df.tolist()
#print(c)

def ilp_model(f,c,pVal):
  n_c=len(c)
  n_f=len(f)
  
  #code for the mathematical model
  model = p.LpProblem('P-Median', p.LpMinimize)
  
  #we create y as the list of decisional binary variables acrive if we use the corrisponding facility
  y = [0]*n_f
  for j in range(n_f):
    #nameV is the name of a specific variable for example (y4)
    nameV="y"+str(j);
    y[j] = p.LpVariable(name=nameV,cat=p.LpBinary)
   
  #we create x as the matrix of decisional binary variables
  x = [0]*n_c
  for i in range(n_c):
    x[i]=[0]*n_f
    for j in range(n_f):    
      nameV="x"+str(i)+"_"+str(j);
      x[i][j] = p.LpVariable(name=nameV,cat=p.LpBinary)
  
 
  #objective function
  obj=p.LpAffineExpression()
  for j in range(n_f):
    obj+=f[j]*y[j] #MULTIPLYING FACILITY COST AND DV OF LOC
  
  for i in range(n_c):
    for j in range(n_f):
      obj+=c[i][j]*x[i][j]
  
  model+=obj
  
  #constraints each customer is served by a single facility
  for i in range(n_c):
    c1=p.LpAffineExpression()
    for j in range(n_f):
      c1+=x[i][j]
    model+=c1==1
  
  #The customer i is served by only an active facility
  for i in range(n_c):
    for j in range(n_f):
      model+=x[i][j]<=y[j]
  
  #set required number of active facilities
  c3=p.LpAffineExpression()
  for j in range(n_f):
    c3+=y[j]
  model+=c3==pVal

  f=open("model.pl","w")
  f.write(str(model))
  f.close()
  

  status = model.solve()  # Solver
  
  print("Status:",p.LpStatus[status])  
  
  obj=p.value(model.objective)
  #print("Objective Value:",obj)
  out=[False]*n_f
  for j in range(n_f):
    if p.value(y[j])>0.5:
      out[j]=True  
  #print (x)
  #print (y) 
  return out

  
#Function that compute the objective function value for a solution
def evaluate_out(f,c,ris):
  n_c=len(c)
  n_f=len(f)
  cost=0
  #Sum of the activation cost of the facilities used.
  for j in range(n_f):
    if ris[j]:
      cost+=f[j]
  
  #We choose the closest facility for each customer
  for i in range(n_c):
    cf=100000000
    for j in range(n_f):
      if ris[j]:
        if cf>c[i][j]:
          cf=c[i][j]
    cost+=cf
    
  return cost;
  

pVal=1 #facilities to active

milp_ris=ilp_model(f,c,pVal)
print(milp_ris)

print("Cost Evaluation MILP:",evaluate_out(f,c,milp_ris))






Status: Optimal
[False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False]
Cost Evaluation MILP: 4334.32403
